In [1]:
import pandas as pd

In [2]:
# Path to the large master CSV file
master_file_path = "../data/processed/master_residue_file.csv"

In [3]:
# Set an appropriate chunk size (adjust based on your RAM; 100,000 rows is a safe start)
chunk_size = 100_000

# Initialize an empty list to collect filtered chunks
positive_chunks = []

# Define the name of the column that indicates binding status
# Replace 'label' with the correct column name if it's different
binding_label_column = 'label'

In [4]:
# Process the file in chunks
for chunk in pd.read_csv(master_file_path, chunksize=chunk_size):
    # Filter only rows where the label is 1 (positive binding residues)
    positive_chunk = chunk[chunk[binding_label_column] == 1].copy()
    
    # Append filtered chunk to the list
    positive_chunks.append(positive_chunk)

# Concatenate all filtered chunks into a single DataFrame
positive_residues_df = pd.concat(positive_chunks, ignore_index=True)

/tmp/ipykernel_239035/1200506279.py:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(master_file_path, chunksize=chunk_size):
/tmp/ipykernel_239035/1200506279.py:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(master_file_path, chunksize=chunk_size):
/tmp/ipykernel_239035/1200506279.py:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(master_file_path, chunksize=chunk_size):
/tmp/ipykernel_239035/1200506279.py:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(master_file_path, chunksize=chunk_size):
/tmp/ipykernel_239035/1200506279.py:2: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk i

In [5]:
positive_residues_df['label'].value_counts()

label
1    648392
Name: count, dtype: int64

In [6]:
ligand_data_path = '../data/processed/BioLiP_positives_residue_level_with_duplicates_1.csv'
positive_ligand_df = pd.read_csv(ligand_data_path)

In [9]:
# Step 2: Select only relevant columns
ligand_columns = [
    'pdb_id',
    'chain_id',
    'pdb_residue_number',
    'ligand_id',
    'ligand_chain',
    'ligand_serial_number',
]

In [10]:
positive_ligand_df.columns
positive_ligand_df = positive_ligand_df[ligand_columns]

In [11]:
positive_ligand_df = positive_ligand_df.drop_duplicates(
    subset=['pdb_id', 'chain_id', 'pdb_residue_number'],
    keep='first'
)

In [12]:
positive_residues_df = positive_residues_df.merge(
    positive_ligand_df,
    on=['pdb_id', 'chain_id', 'pdb_residue_number'],
    how='left'  # 'left' keeps all positive residues, even if some lack ligand info
)

In [14]:
# Define output path
output_path = "../data/processed/master_positive_residue_file.csv"

# Save to CSV
positive_residues_df.to_csv(output_path, index=False)

In [15]:
# Count how many residues are associated with each ligand type
ligand_counts = positive_residues_df['ligand_id'].value_counts(dropna=False).reset_index()

In [16]:
ligand_counts

,ligand_id,count
0,rna,203216
1,peptide,43568
2,dna,32888
3,CLA,31270
4,ZN,24705
...,...,...
5760,HDR,1
5761,SUS,1
5762,OUO,1
5763,TTC,1


In [17]:
# Rename columns for clarity
ligand_counts.columns = ['ligand_id', 'residue_count']

In [18]:
# Define output path for the ligand summary
ligand_summary_path = "../not_necessary_data_and_codes/ligand_residue_counts.csv"

# Save the ligand counts to CSV
ligand_counts.to_csv(ligand_summary_path, index=False)